In [2]:
!pip install transformers accelerate -U

In [1]:
import reservoirtransformers
import math
import os
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import torch.nn.functional as F

In [2]:
from configuration import ReservoirTConfig

configuration = ReservoirTConfig()

configuration.output_size=7
configuration.re_output_size=20
configuration.max_sequence_length=1399
configuration.sequence_length=384
configuration.pred_len=720
configuration.hidden_size=8
configuration.num_attention_heads=4
configuration.hidden_dropout_prob=0.0
configuration.num_hidden_layers=4
configuration.num_reservoirs = 10
configuration.reservoir_size = [30, 15, 20, 25, 30, 35, 40, 45, 50, 50]
configuration.spectral_radius = [0.6, 0.8, 0.55, 0.6, 0.5, 0.4, 0.3, 0.2, 0.81, 0.05]
configuration.sparsity = [0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15]
configuration.leaky = [0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39]
configuration.attention_probs_dropout_prob=0.0
#bert_model = TabularBertForRegression(config=configuration).to("cpu", dtype=float)
model = reservoirtransformers.ReservoirTTimeSeries(config=configuration).to("cpu", dtype=float)


In [3]:
import numpy as np
# prepare data for lstm
from sklearn.preprocessing import StandardScaler
from pandas import read_csv
from pandas import DataFrame
import random
from sklearn.model_selection import train_test_split
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
dataset= read_csv('/content/ETTm2.csv')
dataset=dataset.dropna()
dataset = dataset.drop(['date'], axis = 1)
dataset = dataset.dropna()
#data = dataset.values[0:14000]


y = dataset.OT.values


X = dataset.values

scaler = StandardScaler()
X = scaler.fit_transform(X)



#X=X[1:]

#Reservoir_id = np.array([[0] * len(X[0])] + X[:-1].tolist())
# Create a zero column of shape (100, 1)
'''
zero_col = np.zeros((X.shape[0], 1))

# Concatenate the original array with the zero column along the second axis (columns)
X = np.hstack((X, zero_col))
#X =  dataset.drop(['ate'], axis = 1).values

#X_train, X_test, y_train, y_test =train_test_split(X.values, y, test_size=0.2, shuffle=False)
'''

"\nzero_col = np.zeros((X.shape[0], 1))\n\n# Concatenate the original array with the zero column along the second axis (columns)\nX = np.hstack((X, zero_col))\n#X =  dataset.drop(['ate'], axis = 1).values\n\n#X_train, X_test, y_train, y_test =train_test_split(X.values, y, test_size=0.2, shuffle=False)\n"

In [5]:
X.shape

(69680, 7)

In [4]:
from tqdm.auto import tqdm
# 1. Preprocess the data into the required format
def create_sequences(data, seq_length, pred_length):
    sequences = []
    targets = []
    print('creating seq')
    for i in tqdm(range(len(data) - seq_length - pred_length + 1)):

        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length:i+seq_length+pred_length])
    return torch.tensor(sequences), torch.tensor(targets)

X, y = create_sequences(data=X, seq_length=configuration.sequence_length, pred_length=configuration.pred_len)
# Zeros tensor of shape [16941, 384, 1]
print(X.shape)
zeros = torch.zeros((X.size(0), X.size(1), 1), dtype=X.dtype)

# Concatenate along the last dimension
X = torch.cat((X, zeros), dim=-1)

creating seq


  0%|          | 0/68577 [00:00<?, ?it/s]

torch.Size([68577, 384, 7])


<ipython-input-4-1feea5ab717f>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.tensor(sequences), torch.tensor(targets)


In [7]:
X.shape, y.shape

(torch.Size([69201, 384, 10]), torch.Size([69201, 96, 7]))

In [5]:

batch=64
indices = np.arange(len(X))
barrier = int(len(indices)/batch)*batch
indices = indices[0:barrier]
soft_border = int((configuration.sequence_length/batch))+16

indices = [indices[i:i+batch] for i in range(0, len(indices), batch)]

border1 = int(len(indices)*0.7)
border2 = border1+int(len(indices)*0.1)
border3 = border2+int(len(indices)*0.2)

train_ind = indices[0:border1]
val_ind = indices[border1-soft_border: border2]
test_ind = indices[border2-soft_border: border3]

random.shuffle(train_ind)
random.shuffle(val_ind)
#random.shuffle(test_ind)


X_train = [X[item] for sublist in train_ind for item in sublist]
y_train = [y[item] for sublist in train_ind for item in sublist]

X_val = [X[item] for sublist in val_ind for item in sublist]
y_val = [y[item] for sublist in val_ind for item in sublist]

X_test = [X[item] for sublist in test_ind for item in sublist]
y_test = [y[item] for sublist in test_ind for item in sublist]

#train_indices, test_indices =train_test_split(indices,  test_size=0.2, shuffle=False)
#indices = [item for sublist in indices for item in sublist]

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, tokenized_inputs,  labels=None, pos=None):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels
        self.pos = pos
        self.id_list = None
        self.re = None

    def __len__(self):
        return len(self.tokenized_inputs)

    def __getitem__(self, idx):
        if self.labels is not None:
            return {
                "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
                "labels_ids": torch.tensor(self.labels[idx]),
                #"id": torch.tensor(self.id_list[idx]),  # Include the id directly
                #"reservoir_ids": torch.tensor(self.re[idx]),
            }
        else:
            return {
                "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
            }

# Assuming you have X_train, y_train, X_test, y_test, trainpos, and testpos defined


train_dataset = CustomDataset(X_train, y_train)

test_dataset = CustomDataset(X_test, y_test)

val_dataset = CustomDataset(X_val, y_val)

In [7]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler
class CustomTrainer(Trainer):
    def __init__(self, *args, gradient_accumulation_steps=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.scaler = GradScaler()

    def training_step(self, model, inputs):
        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)

        if self.args.n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training

        loss = loss / self.gradient_accumulation_steps
        self.scaler.scale(loss).backward()

        return loss.detach()


    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training [`~torch.utils.data.DataLoader`].
        Will use no sampler if `train_dataset` does not implement `__len__`, a random sampler (adapted to distributed
        training if necessary) otherwise.
        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset


        loader =  DataLoader(
            train_dataset,
            batch_size=self._train_batch_size,
            drop_last=self.args.dataloader_drop_last,
            shuffle = False,
        )
        return loader

#bert_model = TabularBertForSequenceClassification(config=configuration).to("cpu", dtype=float)


In [8]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import mean_squared_error
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
from sklearn.metrics import r2_score, accuracy_score
import numpy as np

def compute_metrics1(p):

    preds = p.predictions.flatten()
    labels = p.label_ids.flatten()

    r2 = r2_score(labels, preds)
    mse = mean_squared_error(labels, preds)

    return {"r2_score": r2, "mse": mse}

def compute_metrics_classification(p):
    preds = np.argmax(p.predictions , axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)
    return {"accuracy_score": accuracy}

def compute_metrics(p):


    prediction_scores, labels_ids = p
    #print('here')
    #print(prediction_scores)

    mask = labels_ids != 100
    #print(mask)
    masked_predictions = prediction_scores[mask]
    masked_labels = labels_ids[mask]

    mse = mean_squared_error(masked_predictions, masked_labels)
    return {"mse": mse}

training_args = TrainingArguments(
    output_dir='./results_task1',
    num_train_epochs=150,
    label_names=["labels_ids"],
    disable_tqdm = True,
    #label_names=["labels_mask"],
    do_eval=True,
    learning_rate=0.001,
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps = 50,
    save_strategy="steps",
    save_steps=50,

    save_total_limit=2,
    load_best_model_at_end=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics1, #compute_metrics1,#compute_metrics_classification,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=50)]
)

trainer.train()

<ipython-input-6-5df45e9992a5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
<ipython-input-6-5df45e9992a5>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels_ids": torch.tensor(self.labels[idx]),
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.0523, 'learning_rate': 0.0009995549621717847, 'epoch': 0.07}
{'eval_loss': 1.2335270464091228, 'eval_r2_score': -0.800989874865758, 'eval_mse': 1.2335270464091221, 'eval_runtime': 49.0314, 'eval_samples_per_second': 168.382, 'eval_steps_per_second': 2.631, 'epoch': 0.07}


<ipython-input-6-5df45e9992a5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
<ipython-input-6-5df45e9992a5>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels_ids": torch.tensor(self.labels[idx]),


{'loss': 0.966, 'learning_rate': 0.0009991099243435691, 'epoch': 0.13}
{'eval_loss': 0.6462965452913999, 'eval_r2_score': 0.05638588337488515, 'eval_mse': 0.6462965452914005, 'eval_runtime': 48.8686, 'eval_samples_per_second': 168.943, 'eval_steps_per_second': 2.64, 'epoch': 0.13}


<ipython-input-6-5df45e9992a5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
<ipython-input-6-5df45e9992a5>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels_ids": torch.tensor(self.labels[idx]),


...

<ipython-input-6-5df45e9992a5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
<ipython-input-6-5df45e9992a5>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels_ids": torch.tensor(self.labels[idx]),


{'loss': 0.6035, 'learning_rate': 0.0009897641299510458, 'epoch': 1.54}
{'eval_loss': 0.3036625391360481, 'eval_r2_score': 0.5566427505970858, 'eval_mse': 0.30366253913604796, 'eval_runtime': 48.1188, 'eval_samples_per_second': 171.575, 'eval_steps_per_second': 2.681, 'epoch': 1.54}


<ipython-input-6-5df45e9992a5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "inputs_embeds": torch.tensor(self.tokenized_inputs[idx]),
<ipython-input-6-5df45e9992a5>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels_ids": torch.tensor(self.labels[idx]),


{'loss': 0.3334, 'learning_rate': 0.0009893190921228304, 'epoch': 1.6}


KeyboardInterrupt: ignored

In [9]:
model = reservoirtransformers.ReservoirTTimeSeries.from_pretrained("/content/results_task1/checkpoint-1000", config=configuration).to("cuda", dtype=float)


In [10]:
cnt = 0
ln = len(X_test)
y_pred = []
y_test1 = []
while cnt < ln:
    #print(cnt, ln)
    input_ids = torch.stack(X_test[cnt:cnt+batch], dim=0)
    #y_test1 = y_test1 + [k.detach().numpy().flatten() for k in y_test[cnt:cnt+64]]

    output = model(inputs_embeds = input_ids.to(model.device))['logits']
    y_pred = y_pred + list(output.cpu().detach().numpy().reshape(output.size(0), -1))
    #y_test = y_test + labels_ids

    cnt=cnt+batch


In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_test1 = [i.detach().numpy().flatten() for i in y_test]

mse = mean_squared_error(y_test1, y_pred)
mse

0.28511558109858404